In [1]:
import optuna
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import ARIMA
from utilsforecast.evaluation import evaluate

import warnings
import logging

warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("optuna").setLevel(logging.WARNING)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
# Definir las métricas
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

def mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def evaluate(cv_df, metrics):
    results = {}
    for metric in metrics:
        y_true = cv_df['y'].values
        y_pred = cv_df['ARIMA'].values
        results[metric.__name__] = metric(y_true, y_pred)
    return results

In [3]:
# Descargar datos
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'IBM', 'BTC-USD']
Y_df = yf.download(tickers, start='2000-01-01', interval='1wk')['Adj Close']

# Resetear el índice y transformar el DataFrame
Y_df = Y_df.reset_index().melt(id_vars='Date', var_name='unique_id', value_name='y')
Y_df = Y_df.rename(columns={'Date': 'ds'})
Y_df = Y_df[['unique_id', 'ds', 'y']]
Y_df = Y_df.dropna(subset=['y']).reset_index(drop=True)

[*********************100%%**********************]  8 of 8 completed


In [4]:
horizon = 12

In [5]:
def optimize_for_ticker(ticker):
    Y_df_ticker = Y_df[Y_df['unique_id'] == ticker]
    
    # Verifica el número de observaciones
    print(f"Number of observations for {ticker}: {len(Y_df_ticker)}")
    
    # Manejo de NaN
    if Y_df_ticker['y'].isnull().any():
        print(f"NaN values found in {ticker}.")
        return None  # Manejar según sea necesario

    # Definir función objetivo para Optuna
    def objective(trial):
        # Definir el espacio de búsqueda de hiperparámetros
        order = (trial.suggest_int('p', 0, 3), 
                 trial.suggest_int('d', 0, 2), 
                 trial.suggest_int('q', 0, 3))
        seasonal_order = (trial.suggest_int('P', 0, 3), 
                          trial.suggest_int('D', 0, 2), 
                          trial.suggest_int('Q', 0, 3))
        season_length = trial.suggest_int('season_length', 1, min(52, len(Y_df_ticker)))

        # Instanciar el modelo ARIMA
        model = ARIMA(order=order, 
                      seasonal_order=seasonal_order, 
                      season_length=season_length)

        # Instanciar StatsForecast
        sf = StatsForecast(df=Y_df_ticker, models=[model], freq='W', n_jobs=-1)

        cv_df = sf.cross_validation(df=Y_df_ticker, h=horizon, step_size=horizon, n_windows=5)
        cv_df = cv_df.reset_index()

        # Evaluar las predicciones
        evaluation_results = evaluate(cv_df.loc[:, cv_df.columns != 'cutoff'], metrics=[smape, mape, mse, mae, rmse])

        # Obtener SMAPE para la optimización
        smape_value = evaluation_results['smape']
        return smape_value

    # Configurar y ejecutar la optimización con Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=25)

    # Mostrar los mejores parámetros
    best_params = study.best_params
    print(f"Best hyperparameters for {ticker}: {best_params}")
    print(f"Best SMAPE for {ticker}: {study.best_value}")

    return best_params

In [6]:
def create_model_and_predict(ticker, best_params):
    Y_df_ticker = Y_df[Y_df['unique_id'] == ticker]

    # Crear y ajustar el modelo final con los mejores parámetros
    final_model = ARIMA(
        order=(best_params['p'], best_params['d'], best_params['q']),
        seasonal_order=(best_params['P'], best_params['D'], best_params['Q']),
        season_length=best_params['season_length']
    )

    # Instanciar StatsForecast con el modelo final
    sf_final = StatsForecast(df=Y_df_ticker, models=[final_model], freq='W', n_jobs=-1)

    prediction_length = int(len(Y_df_ticker)) - horizon
    Y_df_real = Y_df_ticker[:prediction_length]
    Y_df_test = Y_df_ticker[prediction_length:]

    # Ajustar el modelo
    sf_final.fit(Y_df_real)

    # Realizar predicciones
    forecast = sf_final.forecast(h=horizon)
    
    print(f"Predictions for {ticker}:")
    print(forecast)

    return forecast

In [7]:
def plot_predictions(ticker, forecast):
    Y_df_ticker = Y_df[Y_df['unique_id'] == ticker]
    
    # Graficar
    plt.figure(figsize=(10, 5))

    # Datos reales
    real_series = Y_df_ticker[['ds', 'y']]  
    plt.plot(real_series['ds'], real_series['y'], label='Actual Data', linestyle='-', color='blue', linewidth=2)

    # Predicciones
    forecast_series = forecast[['ds', 'ARIMA']]
    plt.plot(forecast_series['ds'], forecast_series['ARIMA'], label='Prediction', marker='x', color='orange')

    # Configurar el gráfico
    plt.title(f'Predictions vs Actual Data for {ticker}', fontsize=14)  
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Value', fontsize=12)
    plt.legend(fontsize=10)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.grid(visible=True, linestyle='--', alpha=0.5)

    # Mostrar el gráfico
    plt.show()

In [8]:
# Diccionario para almacenar los mejores parámetros
best_params_dict = {}

for ticker in tickers:
    print(f"\nOptimizando para {ticker}...")
    best_params = optimize_for_ticker(ticker)
    
    # Guardamos los mejores parámetros en el diccionario
    best_params_dict[ticker] = best_params


Optimizando para AAPL...
Number of observations for AAPL: 1292


[W 2024-10-07 14:10:00,303] Trial 0 failed with parameters: {'p': 3, 'd': 2, 'q': 1, 'P': 1, 'D': 0, 'Q': 2, 'season_length': 44} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/dt/cxyz36h16ydfv1yg6n6v5fqm0000gn/T/ipykernel_4086/2725803370.py", line 31, in objective
    cv_df = sf.cross_validation(df=Y_df_ticker, h=horizon, step_size=horizon, n_windows=5)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsforecast/core.py", line 1677, in cross_validation
    return super().cross_validation(
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/py

KeyboardInterrupt: 

In [ ]:
import time

# Diccionario para almacenar las predicciones
forecasts_dict = {}

for ticker in tickers:
    print(f"\nCreando el modelo y realizando predicciones para {ticker}...")
    
    predict_start = time.time()
    print(f"Generando predicción para {ticker}")
    
    # Usamos los mejores parámetros almacenados para cada ticker
    best_params = best_params_dict[ticker]
    
    # Creamos el modelo y realizamos las predicciones
    forecast = create_model_and_predict(ticker, best_params)

    predict_end = time.time()

    # Calcular el tiempo de predicción
    predict_time = predict_end - predict_start
    print(f"Tiempo de predicción: {predict_time:.2f} segundos")
    
    # Guardamos las predicciones en el diccionario
    forecasts_dict[ticker] = forecast

In [ ]:
for ticker in tickers:
    print(f"\nGraficando las predicciones para {ticker}...")
    
    # Usamos las predicciones almacenadas para cada ticker
    forecast = forecasts_dict[ticker]
    
    # Graficamos las predicciones
    plot_predictions(ticker, forecast)